In [ ]:
# Setup
import os
from openai import AzureOpenAI, OpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Load env variables
%load_ext dotenv
%dotenv

azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_model_deployment = os.getenv("AZURE_MODEL_DEPLOYMENT")
azure_oai_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")

# Construct the base URL.  
if azure_oai_endpoint:
    if not azure_oai_endpoint.endswith('/'):
        azure_oai_endpoint += '/'
    base_url = azure_oai_endpoint + "openai/v1/"

# To use API key, set this to False.  
use_entra_id = True

In [ ]:
os.system("az login")

In [ ]:
import requests
import base64 
import time

headers = {}

if use_entra_id:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    headers= { "Authorization": f"Bearer {token.token}", "Content-Type": "application/json" }
else:
    headers= { "api-key": azure_oai_key, "Content-Type": "application/json" }


def call_sora_with_text(prompt):
    # 1. Create a video generation job
    create_url = f"{azure_oai_endpoint}/openai/v1/video/generations/jobs?api-version={api_version}"
    body = {
        "prompt": prompt,
        "width": 480,
        "height": 480,
        "n_seconds": 5,
        "model": "sora"
    }
    response = requests.post(create_url, headers=headers, json=body)
    response.raise_for_status()
    print("Full response JSON:", response.json())
    job_id = response.json()["id"]
    print(f"Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{azure_oai_endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version={api_version}"
    status=None
    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)  # Wait before polling again
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        print(f"Job status: {status}")

    # 3. Retrieve generated video 
    if status == "succeeded":
        generations = status_response.get("generations", [])
        if generations:
            print(f"✅ Video generation succeeded.")
            generation_id = generations[0].get("id")
            video_url = f"{azure_oai_endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version={api_version}"
            video_response = requests.get(video_url, headers=headers)
            if video_response.ok:
                output_filename = "output.mp4"
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f'Generated video saved as "{output_filename}"')
        else:
            raise Exception("No generations found in job result.")
    else:
        raise Exception(f"Job didn't succeed. Status: {status}")
    

def call_sora_with_image(prompt, image_path):
    #with open(image_path, "rb") as image_file:
    #    image_data = image_file.read()
    #image_base64 = base64.b64encode(image_data).decode('utf-8')
    #prompt_with_image = f"{prompt}\n![image](data:image/jpeg;base64,{image_base64})"
    #sora_with_text_image(prompt_with_image)

    # 1. Create a video generation job with image inpainting (multipart upload)
    create_url = f"{azure_oai_endpoint}/openai/v1/video/generations/jobs?api-version=preview"

    # Flatten the body for multipart/form-data
    data = {
        "prompt": "A serene forest scene transitioning into autumn",
        "height": str(1080),
        "width": str(1920),
        "n_seconds": str(10),
        "n_variants": str(1),
        "model": "sora",
        # inpaint_items must be JSON string
        "inpaint_items": json.dumps([
            {
                "frame_index": 0,
                "type": "image",
                "file_name": "dog_swimming.jpg",
                "crop_bounds": {
                    "left_fraction": 0.1,
                    "top_fraction": 0.1,
                    "right_fraction": 0.9,
                    "bottom_fraction": 0.9
                }
            }
        ])
    }

    # Replace with your own image file path
    with open("dog_swimming.jpg", "rb") as image_file:
        files = [
            ("files", ("dog_swimming.jpg", image_file, "image/jpeg"))
        ]
        multipart_headers = {k: v for k, v in headers.items() if k.lower() != "content-type"}
        response = requests.post(
            create_url,
            headers=multipart_headers,
            data=data,
            files=files
        )

    if not response.ok:
        print("Error response:", response.status_code, response.text)
        response.raise_for_status()
    print("Full response JSON:", response.json())
    job_id = response.json()["id"]
    print(f"Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{azure_oai_endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version=preview"
    status = None
    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        print(f"Job status: {status}")

    # 3. Retrieve generated video
    if status == "succeeded":
        generations = status_response.get("generations", [])
        if generations:
            generation_id = generations[0].get("id")
            video_url = f"{azure_oai_endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version=preview"
            video_response = requests.get(video_url, headers=headers)
            if video_response.ok:
                output_filename = "output.mp4"
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f'✅ Generated video saved as "{output_filename}"')
        else:
            raise Exception("No generations found in job result.")
    else:
        raise Exception(f"Job didn't succeed. Status: {status}")
    

def call_sora_with_video(prompt, video_path):
    # 1. Create a video generation job with video inpainting (multipart upload)
    create_url = f"{azure_oai_endpoint}/openai/v1/video/generations/jobs?api-version=preview"

    # Flatten the body for multipart/form-data
    data = {
        "prompt": "A serene forest scene transitioning into autumn",
        "height": str(1080),
        "width": str(1920),
        "n_seconds": str(10),
        "n_variants": str(1),
        "model": "sora",
        # inpaint_items must be JSON string
        "inpaint_items": json.dumps([
            {
                "frame_index": 0,
                "type": "video",
                "file_name": "dog_swimming.mp4",
                "crop_bounds": {
                    "left_fraction": 0.1,
                    "top_fraction": 0.1,
                    "right_fraction": 0.9,
                    "bottom_fraction": 0.9
                }
            }
        ])
    }

    # Replace with your own video file path
    with open("dog_swimming.mp4", "rb") as video_file:
        files = [
            ("files", ("dog_swimming.mp4", video_file, "video/mp4"))
        ]
        multipart_headers = {k: v for k, v in headers.items() if k.lower() != "content-type"}
        response = requests.post(
            create_url,
            headers=multipart_headers,
            data=data,
            files=files
        )

    if not response.ok:
        print("Error response:", response.status_code, response.text)
        response.raise_for_status()
    print("Full response JSON:", response.json())
    job_id = response.json()["id"]
    print(f"Job created: {job_id}")

    # 2. Poll for job status
    status_url = f"{azure_oai_endpoint}/openai/v1/video/generations/jobs/{job_id}?api-version=preview"
    status = None
    while status not in ("succeeded", "failed", "cancelled"):
        time.sleep(5)
        status_response = requests.get(status_url, headers=headers).json()
        status = status_response.get("status")
        print(f"Job status: {status}")

    # 3. Retrieve generated video
    if status == "succeeded":
        generations = status_response.get("generations", [])
        if generations:
            generation_id = generations[0].get("id")
            video_url = f"{azure_oai_endpoint}/openai/v1/video/generations/{generation_id}/content/video?api-version=preview"
            video_response = requests.get(video_url, headers=headers)
            if video_response.ok:
                output_filename = "output.mp4"
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                    print(f'✅ Generated video saved as "{output_filename}"')
        else:
            raise Exception("No generations found in job result.")
    else:
        raise Exception(f"Job didn't succeed. Status: {status}")